# 1) Setup

In [1]:
#closeAllConnections() #Remove all the data
rm(list=ls()) #remove all variables

# 2) Packages

In [2]:
libraries <- c("randomizr","maxLik")

In [3]:
for(i in 1:length(libraries)){
  if(!require(package = libraries[i],character.only = TRUE)) 
    install.packages(libraries[i])
}

Loading required package: randomizr
Loading required package: maxLik
Loading required package: miscTools

Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/


# 3) Experiment

In [4]:
set.seed(3) #For reproducibility

## a) Choice scenarios

In [5]:
choiceScenario1 <- c(w1 = 4, v1 = 6, w2 = 6, v2 = 6) 
choiceScenario2 <- c(w1 = 6, v1 = 4, w2 = 6, v2 = 6) 
choiceScenario3 <- c(w1 = 4, v1 = 6, w2 = 6, v2 = 4) 
choiceScenario4 <- c(w1 = 1, v1 = 9, w2 = 9, v2 = 1) 
choiceScenario5 <- c(w1 = 2, v1 = 9, w2 = 8, v2 = 2) 
choiceScenario6 <- c(w1 = 3, v1 = 8, w2 = 8, v2 = 2) 
choiceScenario7 <- c(w1 = 8, v1 = 1, w2 = 2, v2 = 8) 
choiceScenario8 <- c(w1 = 7, v1 = 2, w2 = 2, v2 = 8) 

In [6]:
choiceScenarios <- rbind(s1 = choiceScenario1,s2 = choiceScenario2, s3 = choiceScenario3,s4= choiceScenario4
                         , s5 = choiceScenario5, s6 = choiceScenario6, s7= choiceScenario7, s8 = choiceScenario8)

choiceScenarios <- as.data.frame(choiceScenarios)

choiceScenarios$scenario <- rownames(choiceScenarios); rownames(choiceScenarios) <- NULL

choiceScenarios <- choiceScenarios[,c("scenario","w1","v1","w2","v2")]

choiceScenarios

scenario,w1,v1,w2,v2
s1,4,6,6,6
s2,6,4,6,6
s3,4,6,6,4
s4,1,9,9,1
s5,2,9,8,2
s6,3,8,8,2
s7,8,1,2,8
s8,7,2,2,8


## b) Build dataset

### i) Parameters

In [7]:
cdata <- data.frame() # Choice data
nScenarios <- 8 # Number of choice scenario
nAlternatives <- 2 #Alternatives per scenario
nIndividuals <- 10*72 # Number of individuals (72)
nExperimentalConditions <- 2 #Control and Treatment conditions
nExperimentTypes <- 2 #Numeric or animated

### ii) Create Dataset

In [8]:
#Create 8 scenarios per individual 
for(nIndividual in 1:nIndividuals){
    
    cdata <- rbind(cdata,cbind(individual = nIndividual,choiceScenarios))
}

#Duplicate the observations per individuals as there are two types of experiment, with the same scenarios.
cdata  <- rbind(cbind(cdata, experimentType = "animated"),cbind(cdata, experimentType = "numeric"))

In [9]:
cdata$experimentType <- as.character(cdata$experimentType)

In [10]:
nObservations <- dim(cdata)[1]; nObservations

[1] 11520

In [11]:
head(cdata)

individual,scenario,w1,v1,w2,v2,experimentType
1,s1,4,6,6,6,animated
1,s2,6,4,6,6,animated
1,s3,4,6,6,4,animated
1,s4,1,9,9,1,animated
1,s5,2,9,8,2,animated
1,s6,3,8,8,2,animated


### iii) Random assignment

In [12]:
#library(randomizr)
experimentalCondition <- complete_ra(N = nIndividuals, m = nIndividuals/2) 
experimentalCondition <- ifelse(experimentalCondition == 0, "control","treatment")
nIndividualsConditions <- data.frame(individual = 1:nIndividual,experimentalCondition = experimentalCondition)

In [13]:
cdata <- merge(cdata, nIndividualsConditions, by = "individual")

In [14]:
head(cdata)

individual,scenario,w1,v1,w2,v2,experimentType,experimentalCondition
1,s1,4,6,6,6,animated,control
1,s2,6,4,6,6,animated,control
1,s3,4,6,6,4,animated,control
1,s4,1,9,9,1,animated,control
1,s5,2,9,8,2,animated,control
1,s6,3,8,8,2,animated,control


## c) Choices

### i) General parameters of utility function

In [15]:
simulationParametersNumeric <- list()
simulationParametersAnimated <- list()

#### - Preferences for waiting and in-vehicle times

In [16]:
# Marginal rate of substitution of waiting for travelling
mrs <- 2

# Preferences for waiting and travelling
bv <- -1
bw <-mrs*bv

#Preferences are the same for all experiments
simulationParametersNumeric[["bw"]] <- simulationParametersAnimated[["bw"]] <- bw
simulationParametersNumeric[["bv"]] <- simulationParametersAnimated[["bv"]] <- bv

#### - Time perception functions

+ Numeric condition

In [17]:
#Waiting time
simulationParametersNumeric[["aw"]] <- 1 #Linear parameter
simulationParametersNumeric[["cw"]] <- 0.95 #Exponent 

#In-vehicle time
simulationParametersNumeric[["av"]] <- 1 #Linear parameter
simulationParametersNumeric[["cv"]] <- 0.95 #Exponent 

+ Animated Condition

In [18]:
#Waiting time
simulationParametersAnimated[["aw"]] <- 1 #Linear parameter
simulationParametersAnimated[["cw"]] <- 0.9 #Exponent 

#In-vehicle time
simulationParametersAnimated[["av"]] <- 2 #Linear parameter
simulationParametersAnimated[["cv"]] <- 0.9 #Exponent 

#Treatment condition
simulationParametersAnimated[["awt"]] <- simulationParametersAnimated[["aw"]] #Difference in linear parameter in treatment condition

### ii) Generation of random errors

In [19]:
#The shape (mu: mode) and scale (beta>0) parameters for the error distribution
muError <- 0
betaError <- 1

In [20]:
#Function to generate numbers with gumbel distribution. Source: https://en.wikipedia.org/wiki/Gumbel_distribution

gumbel.dist <- function(mu,beta,n){
  
  numunif <- runif(n,0,1)
  rgumb <- mu-beta*log(-log(numunif))
  
  return(rgumb)
  
}

#Let's assume the same shape and scale parameters for the error distribution for each alternative and accross all choice scenarios
muErrorAnimated <- muError
betaErrorAnimated <- betaError
simulationParametersAnimated[["scale_gumbel"]] = betaErrorAnimated

muErrorNumeric <- muError
betaErrorNumeric <- betaError
simulationParametersAnimated[["scale_gumbel"]] = betaErrorNumeric

#Generation of errors for alternatives from Animated and Numeric experiments
errorAnimated <- data.frame(e1 = gumbel.dist(mu = muErrorAnimated, beta = betaErrorAnimated, nObservations/nExperimentTypes)
                  , e2 = gumbel.dist(mu = muErrorAnimated, beta = betaErrorAnimated, nObservations/nExperimentTypes)
                  )

errorNumeric <- data.frame(e1 = gumbel.dist(mu = muErrorNumeric, beta = betaErrorNumeric, nObservations/nExperimentTypes)
                  , e2 = gumbel.dist(mu = muErrorNumeric, beta = betaErrorNumeric, nObservations/nExperimentTypes)
                 )

In [21]:
head(errorAnimated); head(errorNumeric)

e1,e2
1.3864386,0.3296918
0.5130491,0.1315463
0.8745301,-0.9508476
-0.3658032,-1.5226652
-1.3158759,-0.2073962
3.6950120,-0.2328468


e1,e2
0.1234390,-0.01312055
0.3189870,-0.80958568
-1.3570869,-0.31028734
-1.2213771,-1.15862113
5.7024967,0.23005296
-0.7101695,1.28077568


### iii) Utility computation

In [22]:
utilityComputation <- function(x,beta,error){
  
  aw <- beta[["aw"]]
  bw <- beta[["bw"]]
  cw <- beta[["cw"]]
  
  av <- beta[["av"]]
  bv <- beta[["bv"]]
  cv <- beta[["cv"]]
         
  #Animated experiment 
  if(unique(x$experimentType) == "animated"){ 
    awt <- beta[["awt"]]
    x$V1 <- bw*ifelse(x[,"experimentalCondition"]=="control",aw,aw+awt)*(xAnimated[,"w1"])^cw+bv*av*(x[,"v1"])^cv
    x$V2 <- bw*ifelse(x[,"experimentalCondition"]=="control",aw,aw+awt)*(xAnimated[,"w2"])^cw+bv*av*(x[,"v2"])^cv  
  }

  #Numeric experiment 
  if(unique(x$experimentType) == "numeric"){
    x$V1 <- bw*aw*(x[,"w1"])^cw+bv*av*(x[,"v1"])^cv
    x$V2 <- bw*aw*(x[,"w2"])^cw+bv*av*(x[,"v2"])^cv
  }
  
  x$e1 <- error$e1
  x$e2 <- error$e2
  
  x$U1 <- x$V1+x$e1
  x$U2 <- x$V2+x$e2
    
  return(x)
}


#### - Animated condition

In [23]:
betaAnimated <- simulationParametersAnimated
xAnimated <- subset(cdata,experimentType == "animated")
cdataAnimated <- utilityComputation(x=xAnimated,beta = betaAnimated, e = errorAnimated)

head(cdataAnimated)

individual,scenario,w1,v1,w2,v2,experimentType,experimentalCondition,V1,V2,e1,e2,U1,U2
1,s1,4,6,6,6,animated,control,-16.99591,-20.06301,1.3864386,0.3296918,-15.60947,-19.73332
1,s2,6,4,6,6,animated,control,-16.99591,-20.06301,0.5130491,0.1315463,-16.48286,-19.93146
1,s3,4,6,6,4,animated,control,-16.99591,-16.99591,0.8745301,-0.9508476,-16.12138,-17.94676
1,s4,1,9,9,1,animated,control,-16.44935,-16.44935,-0.3658032,-1.5226652,-16.81515,-17.97201
1,s5,2,9,8,2,animated,control,-18.18148,-16.72817,-1.3158759,-0.2073962,-19.49736,-16.93557
1,s6,3,8,8,2,animated,control,-18.37179,-16.72817,3.6950120,-0.2328468,-14.67678,-16.96102


#### - Numeric condition

In [24]:
betaNumeric <- simulationParametersNumeric
xNumeric <- subset(cdata,experimentType == "numeric")
cdataNumeric <- utilityComputation(x=xNumeric,beta = betaNumeric, error = errorNumeric)

In [25]:
head(cdataNumeric)

,individual,scenario,w1,v1,w2,v2,experimentType,experimentalCondition,V1,V2,e1,e2,U1,U2
9,1,s1,4,6,6,6,numeric,control,-12.95011,-16.45754,0.1234390,-0.01312055,-12.826672,-16.47066
10,1,s2,6,4,6,6,numeric,control,-14.70383,-16.45754,0.3189870,-0.80958568,-14.384839,-17.26713
11,1,s3,4,6,6,4,numeric,control,-12.95011,-14.70383,-1.3570869,-0.31028734,-14.307198,-15.01411
12,1,s4,1,9,9,1,numeric,control,-10.06363,-17.12725,-1.2213771,-1.15862113,-11.285003,-18.28587
13,1,s5,2,9,8,2,numeric,control,-11.92737,-16.35188,5.7024967,0.23005296,-6.224875,-16.12183
14,1,s6,3,8,8,2,numeric,control,-12.88931,-16.35188,-0.7101695,1.28077568,-13.599478,-15.07110


### iv) Simulated choices

In [26]:
cdataNumeric$choice <- with(cdataNumeric, ifelse(U1>U2, 1, 2))
cdataAnimated$choice <- with(cdataAnimated, ifelse(U1>U2, 1, 2))

## d) Estimation

### i) Loglikelihood functions

In [27]:
library(maxLik)

In [28]:
logitProbability <- function(VA,VB){
  
  return(exp(VA)/(exp(VA)+exp(VB)))
  
}

In [29]:
LogitTimePerception <- function(DCMData, logitParameters, constraints, method
                                , simulationParameters, sameCurvature = FALSE, sameTimeWeighting = FALSE){
  
  if(sameCurvature == TRUE)
  { 
    logitParameters[["thetaP"]] <- simulationParameters[["cw"]]
    
    if(c("thetaPW") %in% names(logitParameters)){
      logitParameters <- logitParameters[-which(names(logitParameters) == "thetaPW")] 
    }
    
    if(c("thetaPV") %in% names(logitParameters)){
      logitParameters <- logitParameters[-which(names(logitParameters) == "thetaPV")] 
    }
  }
  
  if(sameTimeWeighting == TRUE)
  { 
    logitParameters[["thetaA"]] <- simulationParameters[["aw"]]
    
    if(c("thetaAW") %in% names(logitParameters)){
      logitParameters <- logitParameters[-which(names(logitParameters) == "thetaAW")] 
    }
    
    if(c("thetaAV") %in% names(logitParameters)){
      logitParameters <- logitParameters[-which(names(logitParameters) == "thetaAV")] 
    }
  }
  
  LogLikTimePerception <<- function(parameters,data = DCMData){
    
    
    
    constant <- as.numeric(ifelse(is.na(parameters["constant"]),0,parameters[["constant"]]))     
    thetaW <- as.numeric(ifelse(is.na(parameters["thetaW"]),simulationParameters[["bw"]]/simulationParameters[["scale_gumbel"]],parameters[["thetaW"]]))
    thetaV <-  as.numeric(ifelse(is.na(parameters["thetaV"]),simulationParameters[["bv"]]/simulationParameters[["scale_gumbel"]],parameters[["thetaV"]]))
    #thetaWeight <- as.numeric(ifelse(is.na(parameters["thetaWeight"]),1/simulationParameters[["betaWeight"]],parameters[["thetaWeight"]]))
    
    thetaAWT <-  as.numeric(ifelse(is.na(parameters["thetaAWT"]),0,parameters[["thetaAWT"]])) 
    
    if(sameCurvature == TRUE){
      thetaPW <-  as.numeric(parameters[["thetaP"]])
      thetaPV <-  as.numeric(parameters[["thetaP"]])
    }
    
    if(sameCurvature == FALSE){
      #sameCurvature = FALSE
      thetaPW <-  as.numeric(ifelse(is.na(parameters["thetaPW"]),simulationParameters[["cw"]],parameters[["thetaPW"]]))
      thetaPV <-  as.numeric(ifelse(is.na(parameters["thetaPV"]),simulationParameters[["cv"]],parameters[["thetaPV"]]))
      
    }
    
    if(sameTimeWeighting == TRUE){
      thetaAW <-  as.numeric(parameters[["thetaA"]])
      thetaAV <-  as.numeric(parameters[["thetaA"]])
    }
    
    if(sameTimeWeighting == FALSE){
      
      #sameTimeWeighting = FALSE
      thetaAW <-  as.numeric(ifelse(is.na(parameters["thetaAW"]),simulationParameters[["aw"]],parameters[["thetaAW"]]))
      thetaAV <-  as.numeric(ifelse(is.na(parameters["thetaAV"]),simulationParameters[["av"]],parameters[["thetaAV"]]))
      
    }
    
    if(unique(data$experimentType) == "numeric"){
      
      V1 <- constant+thetaW*thetaAW*(data$w1)^(thetaPW) + thetaV*thetaAV*(data$v1)^(thetaPV) 
      V2 <- thetaW*thetaAW*(data$w2)^(thetaPW) + thetaV*thetaAV*(data$v2)^(thetaPV)
    }
    
    if(unique(data$experimentType) == "animated"){
      
      #V1 <- constant+thetaW*thetaAW*(data$w1)^(thetaPW) + thetaV*thetaAV*(data$v1)^(thetaPV) 
      #V2 <- thetaW*thetaAW*(data$w2)^(thetaPW) + thetaV*thetaAV*(data$v2)^(thetaPV)
      
      V1 <- thetaW*(data$w1)^(thetaPW)*ifelse(data$experimentalCondition == "control",thetaAW,thetaAW+thetaAWT)+thetaV*thetaAV*(data$v1)^(thetaPV) 
      V2 <- thetaW*(data$w2)^(thetaPW)*ifelse(data$experimentalCondition == "control",thetaAW,thetaAW+thetaAWT)+thetaV*thetaAV*(data$v2)^(thetaPV) 
    }
    
    
    SUM1 <- ifelse(data$choice == 1,1,0)*log(logitProbability(VA = V1, VB = V2))
    SUM2 <- ifelse(data$choice == 2,1,0)*log(logitProbability(VA = V2, VB = V1))
    
    return(SUM1+SUM2)
  }
  
  
  MLParameters <- maxLik(logLik = LogLikTimePerception, start = logitParameters,method = method)
  
  OI <- solve(MLParameters$hessian)
  se <- sqrt(abs(diag(OI)))
  ttest <- MLParameters$estimate/se
  
  resultsML <- cbind(estimate = MLParameters$estimate, se = se, ttest = ttest)
  
  return(list(resultsML,MLParameters))
}

### ii) Parameters recovery

#### - Numeric condition (Control and Treatment)

+ The ratio of the preferences for waiting and travel times are somewhat insensitive to the value of the exponents of the time perception functions provided that the exponents equal for waiting and in-vehicle times 

    * Real exponents

In [30]:
#Control and treatment conditions
simulationParametersNumeric1 <- c(bv = bv, bw = bw, aw = 1, av = 1
                                  , cw = simulationParametersNumeric[["cw"]], cv = simulationParametersNumeric[["cv"]]
                                  , scale_gumbel = betaErrorNumeric) 

logitParametersNumeric1 <- c(thetaW = 0, thetaV = 0, thetaT= NULL,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = NULL, thetaAW = NULL)#, thetaWeight = 1)#, thetaWeight = 1


In [31]:
DCMNumeric1 <- LogitTimePerception(DCMData = subset(cdataNumeric, experimentalCondition == "control")
                                , logitParameters = logitParametersNumeric1, method = "BHHH"
                                , simulationParameters = simulationParametersNumeric1
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMNumeric1

[[1]]
        estimate         se     ttest
thetaW -2.103277 0.09535022 -22.05843
thetaV -1.092681 0.07928612 -13.78149

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 7 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -476.6273 (2 free parameter(s))
Estimate(s): -2.103277 -1.092681 


In [32]:
ratioWV <- DCMNumeric1[[1]]["thetaW","estimate"]/DCMNumeric1[[1]]["thetaV","estimate"]; ratioWV
simulationParametersNumeric[["bw"]]/simulationParametersNumeric[["bv"]]

[1] 1.924877

[1] 2

    * Other exponents

In [33]:
simulationParametersNumeric2 <- simulationParametersNumeric1

simulationParametersNumeric2[["cw"]] <- 0.75
simulationParametersNumeric2[["cv"]] <- 0.75

logitParametersNumeric2 <- c(thetaW = 0, thetaV = 0,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = NULL, thetaAW = NULL)


In [34]:
DCMNumeric2 <- LogitTimePerception(DCMData = subset(cdataNumeric, experimentalCondition == "control")
                                , logitParameters = logitParametersNumeric2, method = "BHHH"
                                , simulationParameters = simulationParametersNumeric2
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMNumeric2

[[1]]
        estimate        se     ttest
thetaW -3.646847 0.1659910 -21.97015
thetaV -1.884544 0.1378194 -13.67401

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 7 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -478.8431 (2 free parameter(s))
Estimate(s): -3.646847 -1.884544 


In [35]:
ratioWV <- DCMNumeric2[[1]]["thetaW","estimate"]/DCMNumeric2[[1]]["thetaV","estimate"]; ratioWV
simulationParametersNumeric[["bw"]]/simulationParametersNumeric[["bv"]]

[1] 1.935135

[1] 2

+ If the exact values of the preferences parameters are known, then it is possible to recover the exponent of the time perception function

In [36]:
simulationParametersNumeric3 <- simulationParametersNumeric1
errorScaleFactor <- 1
simulationParametersNumeric3[["bw"]] <- simulationParametersNumeric1[["bw"]]*errorScaleFactor
simulationParametersNumeric3[["bv"]] <- simulationParametersNumeric1[["bv"]]*errorScaleFactor


logitParametersNumeric3 <- c(thetaW = NULL, thetaV = NULL ,constant = NULL
                      , thetaPW = 1,thetaPV = 1,thetaAV = NULL, thetaAW = NULL)


DCMNumeric3 <- LogitTimePerception(DCMData = subset(cdataNumeric, experimentalCondition == "control")
                                , logitParameters = logitParametersNumeric3, method = "BHHH"
                                , simulationParameters = simulationParametersNumeric3
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMNumeric3

[[1]]
         estimate         se    ttest
thetaPW 0.9709056 0.01705319 56.93394
thetaPV 0.9867217 0.02742628 35.97724

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 4 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -476.4901 (2 free parameter(s))
Estimate(s): 0.9709056 0.9867217 


In [37]:
simulationParametersNumeric[["cw"]];simulationParametersNumeric[["cv"]]

[1] 0.95

[1] 0.95

+ The value of the exponents are sensitive to the scale parameter of the error term. 

In [38]:
simulationParametersNumeric4 <- simulationParametersNumeric1
errorScaleFactor <- 0.5 # scale factor equals to 0.5
simulationParametersNumeric4[["bw"]] <- simulationParametersNumeric1[["bw"]]*errorScaleFactor
simulationParametersNumeric4[["bv"]] <- simulationParametersNumeric1[["bv"]]*errorScaleFactor


logitParametersNumeric4 <- c(thetaW = NULL, thetaV = NULL ,constant = NULL
                      , thetaPW = 1,thetaPV = 1,thetaAV = NULL, thetaAW = NULL)


DCMNumeric4 <- LogitTimePerception(DCMData = subset(cdataNumeric, experimentalCondition == "control")
                                , logitParameters = logitParametersNumeric4, method = "BHHH"
                                , simulationParameters = simulationParametersNumeric4
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMNumeric4


[[1]]
        estimate         se    ttest
thetaPW 1.245065 0.01860257 66.92976
thetaPV 1.258900 0.02997400 41.99974

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 5 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -475.7543 (2 free parameter(s))
Estimate(s): 1.245065 1.2589 


+ It is only possible to recover the ratio between the exponents of the time perception function

In [39]:
ratioPWPV <- DCMNumeric4[[1]]["thetaPW","estimate"]/DCMNumeric4[[1]]["thetaPV","estimate"]; ratioPWPV
simulationParametersNumeric[["cw"]]/simulationParametersNumeric[["cv"]]

[1] 0.9890103

[1] 1

### - Animated Condition (Control: Unknown Waiting Time)

- Previous literature on time perception have found values of the exponents close to 0.9. If the linear parameters of the time perception function are different for waiting and in-vehicles times are different, however, the preferences for waiting and in-vechile times will not be properly recovered

In [40]:
#Lets assume that the scale parameters of the time perception function are equal (e.g. aw = 1, av = 1)

simulationParametersAnimatedControl1 <- c(bv = bv, bw = bw, aw = 1, av = 1, cw = 0.5, cv = 0.5,awt = 0
                                          , scale_gumbel = betaErrorAnimated)


logitParametersAnimatedControl1 <- c(thetaW = 0, thetaV = 0, thetaT= NULL,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAWT = NULL, thetaAV = NULL, thetaAW = NULL)#, thetaWeight = 1)#, thetaWeight = 1


DCMAnimatedControl1 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "control")
                                                     , logitParameters = logitParametersAnimatedControl1, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedControl1
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedControl1

[[1]]
        estimate        se     ttest
thetaW -5.769916 0.2559439 -22.54368
thetaV -5.788429 0.2323270 -24.91501

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 6 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -1336.151 (2 free parameter(s))
Estimate(s): -5.769916 -5.788429 


In [41]:
ratioWV <- DCMAnimatedControl1[[1]]["thetaW","estimate"]/DCMAnimatedControl1[[1]]["thetaV","estimate"]; ratioWV
simulationParametersAnimated[["bw"]]/simulationParametersAnimated[["bv"]]

[1] 0.9968018

[1] 2

+ Let's fix the preferences for waiting and in-vehicle times, so we can properly recover the scale parameter of the time perception function for waiting and in-vehicle times.

In [42]:
simulationParametersAnimatedControl2 <- simulationParametersAnimatedControl1 

simulationParametersAnimatedControl2[["cw"]] <- simulationParametersAnimated[["cw"]]
simulationParametersAnimatedControl2[["cv"]] <- simulationParametersAnimated[["cv"]]

simulationParametersAnimatedControl2[["bw"]] <- DCMNumeric2[[1]]["thetaW","estimate"]
simulationParametersAnimatedControl2[["bv"]] <- DCMNumeric2[[1]]["thetaV","estimate"]

logitParametersAnimatedControl2 <- c(thetaW = NULL, thetaV = NULL, thetaT= NULL ,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAWT = NULL, thetaAV = 1, thetaAW = 1)

DCMAnimatedControl2 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "control")
                                                     , logitParameters = logitParametersAnimatedControl2, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedControl2
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedControl2

[[1]]
         estimate         se    ttest
thetaAV 1.0211052 0.04100734 24.90055
thetaAW 0.5271709 0.02319161 22.73111

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 13 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -1318.044 (2 free parameter(s))
Estimate(s): 1.021105 0.5271709 


In [43]:
ratioAWAV <- DCMAnimatedControl2[[1]]["thetaAW","estimate"]/DCMAnimatedControl2[[1]]["thetaAV","estimate"]; ratioAWAV
simulationParametersAnimated[["aw"]]/simulationParametersAnimated[["av"]]

[1] 0.5162748

[1] 0.5

+ The ratio between the scale parameters of the time perception functions is unsensitive to the value of the exponents of the time perception function or the scale of the preference parameters.

In [44]:
simulationParametersAnimatedControl3 <- simulationParametersAnimatedControl2

factorExponents <- 0.5
simulationParametersAnimatedControl3[["cw"]] <- simulationParametersAnimated$cw*factorExponents
simulationParametersAnimatedControl3[["cv"]] <- simulationParametersAnimated$cv*factorExponents

errorScaleParameter <- 0.8
simulationParametersAnimatedControl3[["bw"]] <- DCMNumeric2[[1]]["thetaW","estimate"]*errorScaleParameter
simulationParametersAnimatedControl3[["bv"]] <- DCMNumeric2[[1]]["thetaV","estimate"]*errorScaleParameter

logitParametersAnimatedControl3 <- c(thetaW = NULL, thetaV = NULL,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = 1, thetaAW = 1)

DCMAnimatedControl3 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "control")
                                                     , logitParameters = logitParametersAnimatedControl3, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedControl3
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedControl3

[[1]]
        estimate        se    ttest
thetaAV 4.508365 0.1805021 24.97680
thetaAW 2.319792 0.1028907 22.54618

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 6 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -1341.603 (2 free parameter(s))
Estimate(s): 4.508365 2.319792 


In [45]:
ratioAWAV <- DCMAnimatedControl3[[1]]["thetaAW","estimate"]/DCMAnimatedControl3[[1]]["thetaAV","estimate"]; ratioAWAV
simulationParametersAnimated[["aw"]]/simulationParametersAnimated[["av"]]

[1] 0.5145529

[1] 0.5

### - Animated Condition (Treatment: Known Waiting Time)

+ Let's assume that the scale parameter of the error term is the same for both experiments. 
The parameters associated to in-vehicle time can be assumed the same in the control and treatment conditions. 

In [46]:
simulationParametersAnimatedTreatment1 <- simulationParametersAnimated


#simulationParametersAnimatedTreatment1[["aw"]] <- simulationParametersAnimated[["aw"]] 
#simulationParametersAnimatedTreatment1[["av"]] <- simulationParametersAnimated[["av"]] 

simulationParametersAnimatedTreatment1[["aw"]] <- DCMAnimatedControl3[[1]]["thetaAW","estimate"]
simulationParametersAnimatedTreatment1[["av"]] <- DCMAnimatedControl3[[1]]["thetaAV","estimate"]


simulationParametersAnimatedTreatment1[["bw"]]<- simulationParametersAnimated[["bw"]]
simulationParametersAnimatedTreatment1[["bv"]] <- simulationParametersAnimated[["bv"]] 

#simulationParametersAnimatedTreatment1[["bw"]]<- DCMNumeric2[[1]]["thetaW","estimate"]
#simulationParametersAnimatedTreatment1[["bv"]] <- DCMNumeric2[[1]]["thetaV","estimate"]

simulationParametersAnimatedTreatment1[["cw"]] <- simulationParametersAnimated[["cw"]]
simulationParametersAnimatedTreatment1[["cv"]] <- simulationParametersAnimated[["cv"]]


logitParametersAnimatedTreatment1 <- c(thetaW = NULL, thetaV = NULL,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = NULL, thetaAW = NULL, thetaAWT=0)


DCMAnimatedTreatment1 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "treatment")
                                                     , logitParameters = logitParametersAnimatedTreatment1, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedTreatment1
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedTreatment1

[[1]]
         estimate         se    ttest
thetaAWT 1.130355 0.07335663 15.40904

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 13 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -172.8304 (1 free parameter(s))
Estimate(s): 1.130355 


+ The ratio between the linear parameters of waiting and in-vehicle times in the time perception function in the treatment condition can be properly recovered.

In [47]:
DCMAnimatedTreatment1[[1]]["thetaAWT","estimate"]/DCMAnimatedControl3[[1]]["thetaAW","estimate"]
simulationParametersAnimatedTreatment1[["awt"]]/simulationParametersAnimatedTreatment1[["aw"]]

[1] 0.4872657

[1] 0.4310731

+ The difference between the control and treatment condition might be also given by a higher sensitivity of participants to waiting time. That is more an effect over the preference than the time perception function (Antonides et al., 2006)

In [55]:
simulationParametersAnimatedTreatment2 <- simulationParametersAnimatedTreatment1

simulationParametersAnimatedTreatment2[["awt"]]<- 0

logitParametersAnimatedTreatment2 <- c(thetaW = 1, thetaV = NULL,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = NULL, thetaAW = NULL, thetaAWT=NULL)

#logitParametersAnimatedTreatment1 <- logitParametersAnimatedControl1

DCMAnimatedTreatment2 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "treatment")
                                                     , logitParameters = logitParametersAnimatedTreatment2, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedTreatment2
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedTreatment2

[[1]]
        estimate         se     ttest
thetaW -2.974554 0.06324413 -47.03288

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 44 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -172.8304 (1 free parameter(s))
Estimate(s): -2.974554 


+ It is not possible to isolate the effect in the scale parameter of perception or the preference. But in any case, the new ratio between waiting and in-vehicle times is given below

In [56]:
ratioWV <- DCMAnimatedTreatment2[[1]]["thetaW","estimate"]/simulationParametersAnimatedTreatment2[["bv"]]; ratioWV
simulationParametersAnimated[["bw"]]/simulationParametersAnimated[["bv"]]

[1] 2.974554

[1] 2

+ If the effect of time perception is not controlled, the ratio between the preferences for waiting and in-vechile times in the numeric and animated experiment are similar, but just for luck

In [60]:
simulationParametersAnimatedTreatment3 <- simulationParametersAnimatedTreatment2

simulationParametersAnimatedTreatment3[["cw"]]<- 1
simulationParametersAnimatedTreatment3[["cv"]]<- 1

simulationParametersAnimatedTreatment3[["aw"]]<- 1
simulationParametersAnimatedTreatment3[["av"]]<- 1

logitParametersAnimatedTreatment3 <- c(thetaW = 1, thetaV = 1,constant = NULL
                      , thetaPW = NULL,thetaPV = NULL, thetaAV = NULL, thetaAW = NULL, thetaAWT=NULL)


DCMAnimatedTreatment3 <- LogitTimePerception(DCMData = subset(cdataAnimated, experimentalCondition == "treatment")
                                                     , logitParameters = logitParametersAnimatedTreatment3, method = "BHHH"
                                , simulationParameters = simulationParametersAnimatedTreatment3
                                , sameCurvature = FALSE, sameTimeWeighting = FALSE)
DCMAnimatedTreatment3

[[1]]
        estimate        se     ttest
thetaW -3.227623 0.1734516 -18.60821
thetaV -1.585296 0.1304079 -12.15644

[[2]]
Maximum Likelihood estimation
BHHH maximisation, 9 iterations
Return code 2: successive function values within tolerance limit
Log-Likelihood: -129.9153 (2 free parameter(s))
Estimate(s): -3.227623 -1.585296 


In [61]:
ratioWV <- DCMAnimatedTreatment3[[1]]["thetaW","estimate"]/DCMAnimatedTreatment3[[1]]["thetaV","estimate"]; ratioWV
simulationParametersAnimated[["bw"]]/simulationParametersAnimated[["bv"]]

[1] 2.035975

[1] 2